In [ ]:
%matplotlib inline



# Write/Load and Upload/Download a Result File
DPF-Core can upload files to and download files from the server machine.

This example shows how to write and upload files on the server machine and then
download them back on the client side. The resulting fields container is exported
in CSV format.


Load a model from the DPF-Core examples:
``ansys.dpf.core`` module.



In [ ]:
from ansys.dpf import core 
from ansys.dpf.core import examples


server_file_path = core.upload_file_in_tmp_folder(examples.simple_bar)
model = core.Model(server_file_path)
mesh = model.metadata.meshed_region

# Get and Plot the Fields Container for the Result
Get the fields container for the result and plot it so you can compare it later:



In [ ]:
displacement_operator = model.results.displacement()
fc_out = displacement_operator.outputs.fields_container()
mesh.plot(fc_out)

# Export Result
Get the fields container for the result and export it in the CSV format:



In [ ]:
file_path = "simple_bar_fc.csv"

export_csv_operator = core.operators.serialization.field_to_csv()
export_csv_operator.inputs.field_or_fields_container.connect(fc_out)
export_csv_operator.inputs.file_path.connect(file_path)
export_csv_operator.run()

Download csv file from server

In [ ]:
import os
local_file_path = os.path.join("/tmp", "simple_bar_fc.csv")
core.download_file(file_path, local_file_path)

Navigating to File > New > Terminal and running `ls /tmp/` should show the file `simple_bar_fc.csv` listed.

# Upload CSV Result File
Upload the file ``simple_bar_fc.csv`` on the server side.
Here, :func:`upload_file_in_tmp_folder` is used because
it is assumed that the server machine architecture is unknown.
However, when the server file path is known, :func:`upload_file`
can be used.



In [ ]:
server_file_path = core.upload_file_in_tmp_folder(local_file_path)
print(server_file_path)

# Remove file to avoid polluting.
os.remove(local_file_path)

# Download CSV Result File
Download the file ``simple_bar_fc.csv``:



In [ ]:
downloaded_client_file_path = os.path.join("/tmp", "simple_bar_fc_downloaded.csv")
core.download_file(server_file_path, downloaded_client_file_path)

Navigating to File > New > Terminal and running `ls /tmp/` should show the file `simple_bar_fc_downloaded.csv` listed.

# Load CSV Result File as Operator Input
Load the fields container contained in the CSV file as an operator input:



In [ ]:
my_data_sources = core.DataSources(server_file_path)
import_csv_operator = core.operators.serialization.csv_to_field()
import_csv_operator.inputs.data_sources.connect(my_data_sources)
downloaded_fc_out = import_csv_operator.outputs.fields_container()
mesh.plot(downloaded_fc_out)

# Remove file to avoid polluting.
os.remove(downloaded_client_file_path)

# Make Operations Over the Imported Fields Container
Use this fields container:



In [ ]:
min_max_op = core.operators.min_max.min_max_fc()
min_max_op.inputs.fields_container.connect(downloaded_fc_out)
min_field = min_max_op.outputs.field_min()
min_field.data